In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Importing API key
from api_keys import g_key


In [2]:
path_file = "../WeatherPy/output_data_file"
data_file = pd.read_csv(path_file)
data_file.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Thompson,90,CA,1583681837,56,55.74,-97.86,-26.00,6.20
1,Avarua,100,CK,1583681965,94,-21.21,-159.78,25.00,2.10
2,Rikitea,100,PF,1583681966,82,-23.12,-134.97,26.07,4.92
3,Hobart,90,AU,1583681864,71,-42.88,147.33,12.00,2.60
4,Provideniya,16,RU,1583681968,95,64.38,-173.30,-22.03,1.93


In [30]:
#Configuring gmaps

gmaps.configure(api_key=g_key)

coordinates = data_file[["Lat","Lng"]]
humidity = data_file["Humidity"]

# Customize the size of the figure
figure_layout = {
    'width': '950px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
# Plotting Heatmap
fig = gmaps.figure(layout =figure_layout)

# Creating heat layer
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, 
                                 dissipating=False,max_intensity=150,
                                 point_radius=3)


# Adding layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

In [ ]:
#Create new DataFrame fitting weather criteria
#Narrow down the cities to fit weather conditions.
#Drop any rows will null values.
#Narrow down the DataFrame to find your ideal weather condition. For example:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.
#Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

